In [1]:
import re,requests,sys
from bs4 import BeautifulSoup
from collections import defaultdict
from datetime import datetime
import pandas as pd

In [3]:
# def get_proxies(proxy_url=None):
    
#     if proxy_url is None:
#         proxy_url = 'https://www.sslproxies.org/'
#     address_regex = r'[0-9]+(?:\.[0-9]+){3}'
#     address_regex = re.compile(address_regex)
    
#     r = requests.get(proxy_url)
#     soup = BeautifulSoup(r.content, 'html.parser')
    
#     ips = [x.text for x in soup.findAll('td') if address_regex.search(x.text)]
#     ports = [x.text for x in soup.findAll('td') if x.text.isnumeric()]
#     proxies = [':'.join(x) for x in zip(ips,ports)]
#     return proxies

def get_proxies(proxy_url=None):
    
    proxy_url = 'https://www.sslproxies.org/'
    r = requests.get(proxy_url)

    soup = BeautifulSoup(r.content, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')
    
    proxies = list()
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append(row.find_all('td')[:2])
    f = lambda x: x[0].string+x[1].string

    return list(map(f,proxies))


def send_request(url,proxy):
    
    proxy_name = f'http://{proxy}'
    proxy = {'https':proxy_name}
    
    try:
        response = requests.get(url,proxies=proxy,timeout=3)
    except Exception:
        print("Ooops there was an error")
        return None
    
    if not response.ok:
        print('Server Response:',response.status_code)
    else:
        soup = BeautifulSoup(response.content,'html.parser')
    return soup


def find_item(listing,find_key,find_attrs):
    try:
        item = listing.find(find_key,attrs=find_attrs)
        item = str(item.find(text=True, recursive=True))
    except:
        item = None
    return item
    
    
def ScrapeNewListing(soup):
    
    data_dict = defaultdict(list)
    listings = soup.findAll('li')
    
    for listing in listings:
        prod_name=" "
        prod_price = " "
        for name in listing.find_all('h3', attrs={'class':"s-item__title"}):
            if(str(name.find(text=True, recursive=False))!="None"):
                prod_name=str(name.find(text=True, recursive=False))
                #item_name.append(prod_name)

                data_dict['listing name'].append(prod_name)

        if(prod_name!=" "):
            price = find_item(listing,'span',{'class':'s-item__price'})
            data_dict['price'].append(price)

            postage = find_item(listing,'span',{'class':'s-item__shipping s-item__logisticsCost'})
            data_dict['postage'].append(postage)

            listing_date = find_item(listing,'span',{'class':'s-item__dynamic s-item__listingDate'})
            data_dict['listing_date'].append(listing_date)

            country = find_item(listing,'span',{'class':'s-item__location s-item__itemLocation'})
            data_dict['country'].append(country)

            buying_option = find_item(listing,'span',{'class':'s-item__purchase-options-with-icon'})
            data_dict['buying_option'].append(buying_option)

            # Maybe link this to the get request instead of creating a timestamp here
            scraping_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            data_dict['scraping_time'].append(scraping_time)
            
    return pd.DataFrame(data_dict),'new-listings'

def save_json(obj,filename):
    with open(filename,'w') as file:
        json.dump(obj,file)

In [5]:
requests.request(method='get',
                 url='https://www.ebay.co.uk/sch/i.html?_from=R40&_nkw=warhammer&_sacat=0&LH_BIN=1',
                 proxies={'https':"http:109.105.205.232:59152"},timeout=5)

ConnectTimeout: HTTPSConnectionPool(host='www.ebay.co.uk', port=443): Max retries exceeded with url: /sch/i.html?_from=R40&_nkw=warhammer&_sacat=0&LH_BIN=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f2a3c7de130>, 'Connection to 109.105.205.232 timed out. (connect timeout=5)'))

In [6]:
import time

class proxy_runner:
    
    def __init__(self,proxies,url,scraper):
        self.proxies = proxies
        self.proxy = next(self.proxies)
        self.url = url
        self.scraper = scraper
        
    def next_proxy(self):
        ''' Fetch the next proxy. Once empty raise an Exeption to end the process.'''
        try:
            next_proxy = next(self.proxies)
            self.proxy = next_proxy
            return next_proxy
        
        except:
            #raise Exception('Proxy list has been expended')
            print('Proxy list expended. Now exiting process.')
            sys.exit()
    
    def make_request(self,**kwargs):
        ''' Make a Request. If unsuccesful try the next proxy.'''
        while self.proxy!=None:
            try:
                response = requests.request(**kwargs)
                return response
            
            except:
                self.next_proxy()
            
            
    def run(self):
        
        scrape = True
        
        counter = 0
        while scrape:
            
            next_proxy = False
            counter+=1
            
            print(f'Sending Request with proxy {self.proxy}... ')
            
            try:
                #proxy = f'http://{self.proxy}'
                #proxy_add = {'https':proxy}
                #response = requests.request('get', self.url, proxies=proxy_add, timeout=5)
                response = self.make_request(method='get',url=self.url,proxies={'https':self.proxy},timeout=5)
                
                if response.status_code==200:
                    #print("Success!")
                    
                    soup = BeautifulSoup(response.content, 'html.parser')
                    output_df,name = self.scraper(soup)
                    filename = name+'-'+datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
                    output_df.to_json(f'{filename}.json',orient='records')
                    
                    print('Succes Sleeping for 30min now..')
                    return soup
                    break
                    # Sleep for 30 minutes before next request
                    time.sleep(1800)
                else:
                    next_proxy=True
            except Exception as e:
                print(f'\t failed with Error code \"{e}\"')
                next_proxy=True
                
            if next_proxy:
                try:
                    check = self.next_proxy()
                except:
                    scrape = False
                    print("Tried all proxies. Now exiting.")
                    
            time.sleep(1)

In [7]:
proxies = get_proxies() 
proxy_generator = (proxy for proxy in proxies)

runner = proxy_runner(proxies=proxy_generator,
                     url="https://www.ebay.co.uk/sch/i.html?_from=R40&_nkw=warhammer&_sacat=0&LH_BIN=1",
                     scraper=ScrapeNewListing)
soup = runner.run()

Sending Request with proxy 69.63.170.743128... 


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Proxy list expended. Now exiting process.
Traceback (most recent call last):
  File "/home/cemlyn/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/home/cemlyn/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/cemlyn/anaconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cemlyn/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 667, in urlopen
    self._prepare_proxy(conn)
  File "/home/cemlyn/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 932, in _prepare_proxy
    conn.conn

TypeError: object of type 'NoneType' has no len()

In [7]:
with open('ebay_response.html','w') as file:
    file.write(soup.decode())

In [43]:
with open('ebay_response.html','r') as file:
     listing_html = file.read()

In [44]:
listing_soup = BeautifulSoup(listing_html,'html.parser')

In [47]:
def get_listings(soup):
    listings = soup.findAll('li',attrs={'class':'s-item s-item--watch-at-corner'})
    listings_found = dict()
    for listing in listings:
        
        listing_url = listing.find('a',attrs={'class':'s-item__link'}).get('href')
        
        title = listing.find('h3',attrs={'class':'s-item__title'})
        title = str(title.find(text=True, recursive=False))
        
        listings_found[title] = listing_url
        
        # Go into the url and scrape somek extra data
    return listings_found

x = get_listings(listing_soup)

In [2]:
import pandas as pd
pd.DataFrame.from_records('./Data/listings-2021-01-04-19:10:18.json')

,0
0,.
1,/
2,D
3,a
4,t
5,a
6,/
7,l
8,i
9,s
